# Project 2 – Language Generation Using N-Gram Models

## By Derrin Naidoo (2127039)

In [1]:
NAME = "Derrin Naidoo"
STUDENT_NUMBER = "2127039"

## Setup

Note, the code was executed using a Windows 11 machine on a GPU accelerated Jupyter Notebook.

In [2]:
# Package Installations

!pip install os
!pip install numpy
!pip install nltk

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [3]:
# Imports 

import os
import time
import numpy as np

import nltk
import random
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

## Question 1: Read in text files

Note, the variables storing the text files are: **euclid_book**, **monte_script**, **git_source** and **concat_shkspr**.

In [4]:
#Read in euclid elements book
with open('project_2_data/euclid_elements/book.txt', 'r') as file:
    euclid_book = file.read() #.replace('\n'," ")

In [5]:
#Read in monte python scripts
with open('project_2_data/monte_python/scripts.txt', 'r') as file:
    monte_script = file.read() #.replace('\n'," ")

In [6]:
#Read in git source file
with open('project_2_data/git/source.c', 'r', encoding="utf8") as file:
    git_source = file.read() #.replace('\n'," ")

In [7]:
#Read in shakespeare's files
files = []   
filenames = os.listdir('project_2_data/shakespeare/') #Retrieve list of file names from specified directory
    
#For each filename in array
for fname in filenames:
    #Open the file
    file_to_open = 'project_2_data/shakespeare/' + fname 
    with open(file_to_open) as file:
        files.append(file.read()) #Append the file to a list
        
#Concatenate files together into a single variable
concat_shkspr = ''
for file in files:
    concat_shkspr += file
    
    
#TEST CASE: Test if the concatenated shakespeare data is the same length as the sum of individual shakespeare files
totalLength = 0
for file in files:
    totalLength += len(file)
    
assert( len(concat_shkspr) == totalLength )

## Question 2: Concatenate entire dataset

In [8]:
# Variable to store concatenated dataset
concat_data = ''

start = time.time() #Start timer 

# Add book to dataset
concat_data += euclid_book
assert(len(concat_data) == len(euclid_book)) # Length of dataset should be the sum of lengths of book at this point

# Add script to dataset
concat_data += monte_script
assert(len(concat_data) == len(euclid_book) + len(monte_script)) # Length of dataset should be the sum of lengths of book and script at this point

# Add git source to dataset
concat_data += git_source
assert(len(concat_data) == len(euclid_book) + len(monte_script) + len(git_source)) # Length of dataset should be the sum of lengths of book, script and git source at this point

#Add shakespeare to dataset
concat_data += concat_shkspr
assert(len(concat_data) == len(euclid_book) + len(monte_script) + len(git_source) + len(concat_shkspr)) # Length of dataset should be the sum of all individual datasets at this point

end = time.time() #End timer 

#Show time taken to produce concatenated dataset
print(f'Time taken for concatenation: {end-start}s')

Time taken for concatenation: 0.008598566055297852s


In [9]:
print(f'Concatenated dataset contains {len(concat_data)} characters')

Concatenated dataset contains 11261284 characters


## Question 3: Preprocessing

### 3.1 Lowercase Preprocessing

In [10]:
#Takes in a peice of text and converts the entire set to lowercase characters only
def textToLower(text):
    lower_text = '' #Have to copy list so we don't alter the original array parameter
    for i in range(len(text)):
        lower_text += text[i].lower()
    return lower_text

In [11]:
#Test Case: Ensure that the textToLower function works correctly
test = 'AaBbCcDdFf'
assert(textToLower(test) == 'aabbccddff')

test = 'a  B  D  f'
assert(textToLower(test) == 'a  b  d  f')

test = 'sA[]##$FP)'
assert(textToLower(test) == 'sa[]##$fp)')

test = '10A9 4s8G'
assert(textToLower(test) == '10a9 4s8g')

In [12]:
#Create a version of each dataset with only lowercase characters

start = time.time() #Start timer 

lower_euclid_book = textToLower(euclid_book)
assert(len(euclid_book) == len(lower_euclid_book)) #Ensure the number of chars in dataset hasn't changed

lower_monte_script = textToLower(monte_script)
assert(len(monte_script) == len(lower_monte_script)) #Ensure the number of chars in dataset hasn't changed

lower_git_source = textToLower(git_source)
assert(len(git_source) == len(lower_git_source)) #Ensure the number of chars in dataset hasn't changed

lower_concat_shkspr = textToLower(concat_shkspr)
assert(len(concat_shkspr) == len(lower_concat_shkspr)) #Ensure the number of chars in dataset hasn't changed

lower_concat_data = textToLower(concat_data)
assert(len(concat_data) == len(lower_concat_data)) #Ensure the number of chars in dataset hasn't changed

end = time.time() #End timer 

#Show time taken to produce lowercase datasets
print(f'Time taken for lowercase conversion: {end-start}s')

Time taken for lowercase conversion: 26.070250749588013s


### 3.2 Obtain Sentences

In [13]:
#Obtain sentences for each dataset by splitting each lowercase dataset by sequence of '\n\n' characters

start = time.time() #Start timer 

sen_euclid_book = lower_euclid_book.split('\n\n')
print(f'Euclid Elements book has {len(sen_euclid_book)} sentences\n')

sen_monte_script = lower_monte_script.split('\n\n')
print(f'Monte Python scripts has {len(sen_monte_script)} sentences\n')

sen_git_source = lower_git_source.split('\n\n')
print(f'Git Source has {len(sen_git_source)} sentences\n')

sen_concat_shkspr = lower_concat_shkspr.split('\n\n')
print(f'Concatenated Shakespeare has {len(sen_concat_shkspr)} sentences\n')

sen_concat_data = lower_concat_data.split('\n\n')
print(f'Concatenated data has {len(sen_concat_data)} sentences\n')


end = time.time() #End timer 

#Show time taken to produce lowercase datasets
print(f'Time taken for sentence extraction: {end-start}s')

Euclid Elements book has 3014 sentences

Monte Python scripts has 15524 sentences

Git Source has 19444 sentences

Concatenated Shakespeare has 35936 sentences

Concatenated data has 73915 sentences

Time taken for sentence extraction: 0.047324419021606445s


### 3.3 Tokenise Sentences

In [14]:
#Function to return an array of words from a sentence
def tokenisedSentence(sentence, split_char=''):
    if split_char=='':
        words = sentence.split()
    else:
        words = sentence.split(split_char)
    return words

In [15]:
start = time.time() #Start timer 

processed_euclid_book = []
for sentence in sen_euclid_book:
    processed_euclid_book.append(tokenisedSentence(sentence))

processed_monte_script = []
for sentence in sen_monte_script:
    processed_monte_script.append(tokenisedSentence(sentence))

processed_git_source = []
for sentence in sen_git_source:
    processed_git_source.append(tokenisedSentence(sentence))    

processed_concat_shkspr = []
for sentence in sen_concat_shkspr:
    processed_concat_shkspr.append(tokenisedSentence(sentence))

processed_concat_data = []
for sentence in sen_concat_data:
    processed_concat_data.append(tokenisedSentence(sentence))

end = time.time() #End timer 

#Show time taken to produce tokenised datasets
print(f'Time taken to obtain tokenised datasets: {end-start}s')

Time taken to obtain tokenised datasets: 0.35183024406433105s


In [16]:
# Convert preprocessed datasets to numpy arrays
# Note, after all preprocessing, the dataset variables are renamed to the original name given upon reading files (Question 1)
# and, each sentence is still of type list, but the entire array of sentences is a numpy array 

euclid_book = np.array(processed_euclid_book, dtype=object)
monte_script = np.array(processed_monte_script, dtype=object)
git_source = np.array(processed_git_source, dtype=object)
concat_shkspr = np.array(processed_concat_shkspr, dtype=object)
concat_data = np.array(processed_concat_data, dtype=object)

In [17]:
#Test Case: Tokenised datasets should have the same number of sentences before tokenisation

assert(len(sen_euclid_book) == euclid_book.shape[0])
assert(len(sen_monte_script) == monte_script.shape[0])
assert(len(sen_git_source) == git_source.shape[0])
assert(len(sen_concat_shkspr) == concat_shkspr.shape[0])
assert(len(sen_concat_data) == concat_data.shape[0])

## Question 4: Train Language Model

In [18]:
#Function to return a trained language model based on a certain n_gram and dataset
def trainLanguageModel(n_gram, data):
    
    # Initialise language model based on Maximum Likelihood Estimator with a certain n_gram
    languageModel = MLE(n_gram) 
    
    #Preprocess data into an everygram with <s> and </s> padding
    train, vocab = padded_everygram_pipeline(2, data)
    
    #Fit the language model to the training data and vocabulary
    languageModel.fit(train, vocab)
    
    return languageModel

In [19]:
n_grams = [2,3,4] # Stores n-grams to train
all_datasets = [euclid_book, monte_script, git_source, concat_shkspr, concat_data] # Stores datasets 
dataset_names = ["Euclid Book","Monte Scripts","Git Source","Concatenated Shakespeare","Concatenated Data"] # Dataset names 
trained_lms = [] #Stores all trained n-gram language models

# ============================== Train Language Models ==============================
start = time.time() #Start timer 

#For each dataset
for i in range(len(all_datasets)):
    print(f"========================= Training {dataset_names[i]} Dataset ({i+1}/{len(all_datasets)}) =========================")
    
    #For each n-gram
    for n_gram in n_grams:
        trained_lms.append(trainLanguageModel(n_gram, all_datasets[i])) #Append trained language model to list
        print(f"<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained {n_gram}-Gram Language Model >>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print('\n')


end = time.time() #End timer 
# ===================================================================================

#Show time taken to produce lowercase datasets
print(f'Time taken for language model training: {(end-start)/60}mins')


#Test Case: Ensure we've obtained 15 language models after training
assert(len(trained_lms) == 15)

========================= Training Euclid Book Dataset (1/5) =========================
<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained 2-Gram Language Model >>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained 3-Gram Language Model >>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained 4-Gram Language Model >>>>>>>>>>>>>>>>>>>>>>>>>>>


========================= Training Monte Scripts Dataset (2/5) =========================
<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained 2-Gram Language Model >>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained 3-Gram Language Model >>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained 4-Gram Language Model >>>>>>>>>>>>>>>>>>>>>>>>>>>


========================= Training Git Source Dataset (3/5) =========================
<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained 2-Gram Language Model >>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained 3-Gram Language Model >>>>>>>>>>>>>>>>>>>>>>>>>>>
<<<<<<<<<<<<<<<<<<<<<<<<<<< Trained 4-Gram Lan

#### Brief Description

At this point, we've trained 15 language models and appended them to the **trained_lms** list.<br>

Each N-Gram in the list follows eachother **consequetively** and is in the order: **2-Gram, 3-Gram and 4-Gram**<br>
The order of the list as per each index is as follows:<br><br>

**0,1,2**: Euclid Book Dataset<br>
**3,4,5**: Monte Scripts Dataset<br>
**6,7,8**: Git Source Dataset<br>
**9,10,11**: Concatenated Shakespeare Dataset<br>
**12,13,14**: Concatenated Dataset<br>

## Question 5: Generate Sentences

In [20]:
#Function to formate a sentence from a list into a line of text where words are seperated by empty characters
def formattedSentence(sentence_list):
    text = '' #Initilaise text to empty string
    
    #For each word of the sentence
    for i in range(len(sentence_list)):
        text += sentence_list[i]
        #Add an empty character if it's not the last word of the sentence
        if i+1 < len(sentence_list):
            text += ' '
            
    return text

In [21]:
#Function to generate a sentence given the formatted list of trained language models and set parameters
#A description of parameters if given below:
# words_per_sentence: boolean which specifies the number of words in a generated sentence
# formatted_sentence: boolean which specifies whether the printed sentence is formatted or not
# random_seed: specifies boolean which whether the seed for each sentence should be the same or different
# seeds: list which contains seeds for each sentence that's produced (must be of length NUM_SENTENCES)

def generateSentences(trained_lms, words_per_sentence=10, formatted_sentence = True, random_seed=True, seeds = None):
    
    
    # ===== Constants =====
    NUM_SENTENCES = 30
    
    #Range to generate random numbers
    LOWER_RANGE = 1
    UPPER_RANGE = 1000
    
    # =====================
    
    # ==================== Generating/Assigning Seeds for Sentences ====================

    #Initialise random_seeds array
    sentence_seeds = []
    
    #If we want a random seed to be chosen
    if seeds == None:
        #If we want the same seed for every sentence
        if random_seed == False:
            #Using the same seed for each sentence, but the seed is chosen randomly
            random_num = random.randint(LOWER_RANGE, UPPER_RANGE)
            sentence_seeds = [random_num for i in range(30)]
        
        #If we want a different seed for every sentence (where seeds are not repeated)
        else:
            while len(sentence_seeds) != 30:
                random_num = random.randint(LOWER_RANGE, UPPER_RANGE)
                if random_num not in sentence_seeds:
                    sentence_seeds.append(random_num)
                    
    #If we're using the seeds passed into the function
    else:
        assert(len(seeds) == NUM_SENTENCES) # Must have a seed for every sentence
        sentence_seeds = seeds    
        
    # ==================================================================================

    # Store each language model in a dictionary for the relevent dataset
    dict_data = {}
    dict_data["Euclid Book"] = [trained_lms[i] for i in [0,1,2]]
    dict_data["Monte Scripts"] = [trained_lms[i] for i in [3,4,5]]
    dict_data["Git Source"] = [trained_lms[i] for i in [6,7,8]]
    dict_data["Concatenated Shakespeare"] = [trained_lms[i] for i in [9,10,11]]
    dict_data["Concatenated Data"] = [trained_lms[i] for i in [12,13,14]]
    
    
    # Print Metadata for generating sentences
    print('\033[1;3m= = = = = = = = = = = = = = = = = = = Metadata = = = = = = = = = = = = = = = = = = =\033[0m')
    print(f'Words per sentence: {words_per_sentence}')
    print(f'Number of Sentences: {NUM_SENTENCES}')
    if random_seed == False:
        print(f'Seed: {sentence_seeds[0]}')
    else:
        print(f'Seeds: {sentence_seeds}')
        
    print('\033[1;3m= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = \033[0m')
    
    print('\n')
    

    # ==================== Generating and Printing Sentences ====================
    
    start = time.time() #Start timer
    
    #For each dataset
    for key in dict_data:
        
        print(f'\033[1;3m================================ {key} Sentences ================================\033[0m')
    
        n_gram_counter = 2
        
        #For each language model (each n-gram)
        for lm in dict_data[key]:
            print(f'<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< {n_gram_counter}-Gram >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n')
            start_words = []
            #For each of the 2 sentences
            for i in range(2):
                
                #Choose a start word that's not already chosen for this N-Gram
                start_word = np.random.choice(list(lm.vocab))
                counter = 0
                #Keep generating sentences until a sentence starts with a different start word
                while start_word in start_words and counter < 99999:
                    start_word = np.random.choice(list(lm.vocab))
                    counter += 1 # Alternate escape condition
                start_words.append(start_word)
                
                print(f'(Start Word: {start_word})')
                #Generate a sentence
                generated_sentence = lm.generate(words_per_sentence, text_seed=[start_word], random_seed = sentence_seeds.pop())
                
                #Print sentence
                if formatted_sentence == True:
                    print(formattedSentence(generated_sentence))
                    print('\n')
                else:
                    print(generated_sentence)
                    print('\n')
            n_gram_counter += 1

    end = time.time() #End timer 
    
    # ===========================================================================

    #Show time taken to generate sentences
    print(f'Time taken to generate sentences: {end-start}s')
    

In [22]:
#Generate Sentences
generateSentences(trained_lms, formatted_sentence = False, random_seed = False)

= = = = = = = = = = = = = = = = = = = Metadata = = = = = = = = = = = = = = = = = = =
Words per sentence: 10
Number of Sentences: 30
Seed: 533
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 


================================ Euclid Book Sentences ================================
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 2-Gram >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

(Start Word: thus:-)
['1409f086', '</s>', 'any', 'part', 'between', 'the', 'circle', 'described', 'on', 'this']


(Start Word: english)
['computer,', 'carried', 'as', 'i.', '</s>', 'convex', 'or', 'less', 'than', 'two']


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 3-Gram >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

(Start Word: remainder.)
['</s>', 'present', 'day-the', 'production', 'note', 'b.', '&&', '</s>', 'now', 'the']


(Start Word: ratio.)
['</s>', 'present', 'day-the', 'production', 'note', 'b.', '&&', '</s>', 'now', 'the']


<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 4-Gram >>>>>>>>>>>>>>>>>>>>>>>>>>>>>

## Question 6: Comparative Analysis

We observe through the results that the ability of a language model to generate coherent sentences is dependent on the size of the training corpus as well as the size of the n-gram. The sentences generated by 2-Gram language models sound less coherent which indicates that the model lacks contextual information which would allow the sentences to sound more natural. This lack of context means that the language model tends to generalise its sentences. However, we see that while 2-grams have limited context, the increased size of training corpora allows for more context even amongst lower-sized n-grams. We observe this particularly in the **Concatenated Shakespeare** 2-gram sentences (large training corpus) which sound more coherent compared to the **Euclid Elements book** 2-gram sentences (smaller training corpus). The disparity in training size accounts for this ability to generate natural sentences or lack thereof. Consequently, we observe that in the **Concatenated Shakespeare** 4-gram sentences, the text seems quite natural at first glance due to the combination of large training data and increased contextual information for words. Importantly, if the contexts in training corpora should differ to a large degree, as in the **Concatenated Dataset** case where the training corpus is a combination of all other datasets, the language model aims to fit all the included data which means sentences will either be very general or exhibit bias towards a single dataset. In our case, the datasets contexts are significantly different (i.e. Shakespeare, code, textbook, etc), therefore, we observe that the sentences tend to form bias towards datasets where the start words and subsequent words are uniquely found. In particular, we observe a bias towards the **Git Source** dataset as words in that dataset tend to be uniquely found in there amongst the other datasets which means that these sentences infequently contain words from the other datasets. As a final note, we also observe that occasionally even with different start words, the generated sentences for lower sized n-grams may be identical which infers that the the probability distribution of a word occuring may be equal given different contexts. 